## Generación de los set de entrenamiento y test.

El objetivo de este notebook es generar un set de pruebas con la misma estructura estadística del cual hay que predecir.

Esto nos permitirá luego hacer verificaciones "off-line" de los algoritmos utilizados y los distintos tipos de entrenamiento.

In [1]:
import pandas as pd
import numpy as np

from rutas import *

%matplotlib inline

In [45]:
postulantes = pd.read_csv(RUTA_BASE + 'postulantes.csv')

/usr/local/lib/python3.5/dist-packages/IPython/core/interactiveshell.py:2785: DtypeWarning: Columns (1,2,3,4,5,6,7) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [46]:
postulantes.head()

,idpostulante,Doctorado,Master,Otro,Posgrado,Secundario,Terciario/Técnico,Universitario,fechanacimiento,sexo
0,0z5Dmrd,0,0,0,0,0,0,Graduado,1965-10-22,MASC
1,0z5JW1r,0,0,0,0,0,0,Abandonado,1971-04-09,MASC
2,0z5VvGv,0,0,0,0,Graduado,0,0,1987-10-18,MASC
3,0zB01pE,0,0,0,0,0,0,Abandonado,1989-12-01,MASC
4,0zB026d,0,0,0,0,0,Graduado,0,1978-02-23,FEM


In [47]:
kaggle_avisos = pd.read_csv(RUTA_SET_KAGGLE)
kaggle_postulantes = pd.DataFrame(kaggle_avisos['idpostulante']).drop_duplicates()
kaggle_avisos = pd.DataFrame(kaggle_avisos['idaviso']).drop_duplicates()

In [48]:
kaggle_postulantes.head()

,idpostulante
0,6M9ZQR
1,6v1xdL
2,ezRKm9
3,1Q35ej
4,EAN4J6


In [49]:
len(kaggle_postulantes)

64646

In [50]:
kaggle_postulantes = pd.merge(
    kaggle_postulantes.set_index('idpostulante'), 
    pd.read_csv(RUTA_BASE + 'fiuba_1_postulantes_educacion.csv', usecols=['idpostulante', 'estado']).groupby('idpostulante').agg('count'),
    left_index=True,
    right_index=True,
    how='left'
).rename(columns={'estado': 'hay_datos_educacion'}).reset_index()
kaggle_postulantes['hay_datos_educacion'] = kaggle_postulantes['hay_datos_educacion'].fillna(0).map(lambda x: 1 if x else 0)

kaggle_postulantes = pd.merge(
    kaggle_postulantes, 
    pd.read_csv(RUTA_BASE + 'fiuba_2_postulantes_genero_y_edad.csv'),
    on='idpostulante',
    how='left'
)

kaggle_postulantes['edad'] = kaggle_postulantes['fechanacimiento'].fillna('2018-00-00').map(lambda x: 2018 - int(x.split('-')[0])).map(lambda x: x if 14 < x < 90 else 0)
kaggle_postulantes = kaggle_postulantes.drop(axis=1, labels=['fechanacimiento'])

kaggle_postulantes = pd.merge(
    kaggle_postulantes.set_index('idpostulante'), 
    pd.read_csv(RUTA_BASE + 'fiuba_3_vistas.csv', usecols=['idpostulante', 'idAviso']).groupby('idpostulante').agg('count'),
    left_index=True,
    right_index=True,
    how='left'
).rename(columns={'idAviso': 'cantidad_vistas'})
kaggle_postulantes['cantidad_vistas'] = kaggle_postulantes['cantidad_vistas'].fillna(0)

kaggle_postulantes = pd.merge(
    kaggle_postulantes, 
    pd.read_csv(RUTA_BASE + 'fiuba_4_postulaciones.csv', usecols=['idpostulante', 'idaviso']).groupby('idpostulante').agg('count'),
    left_index=True,
    right_index=True,
    how='left'
).rename(columns={'idaviso': 'cantidad_postulaciones'})
kaggle_postulantes['cantidad_postulaciones'] = kaggle_postulantes['cantidad_postulaciones'].fillna(0)

In [51]:
postulantes['edad'] = postulantes['fechanacimiento'].fillna('0000-00-00').map(lambda x: 2018 - int(x.split('-')[0])).map(lambda x: x if 14 < x < 90 else 0)
postulantes = postulantes.drop(axis=1, labels=['fechanacimiento'])

postulantes = pd.merge(
    postulantes.set_index('idpostulante'), 
    pd.read_csv(RUTA_BASE + 'fiuba_3_vistas.csv', usecols=['idpostulante', 'idAviso']).groupby('idpostulante').agg('count'),
    left_index=True,
    right_index=True,
    how='left'
).rename(columns={'idAviso': 'cantidad_vistas'})
postulantes['cantidad_vistas'] = postulantes['cantidad_vistas'].fillna(0)

postulantes = pd.merge(
    postulantes, 
    pd.read_csv(RUTA_BASE + 'fiuba_4_postulaciones.csv', usecols=['idpostulante', 'idaviso']).groupby('idpostulante').agg('count'),
    left_index=True,
    right_index=True,
    how='left'
).rename(columns={'idaviso': 'cantidad_postulaciones'})
postulantes['cantidad_postulaciones'] = postulantes['cantidad_postulaciones'].fillna(0)

In [52]:
kaggle_postulantes.head()

,hay_datos_educacion,sexo,edad,cantidad_vistas,cantidad_postulaciones
idpostulante,,,,,
6M9ZQR,1,FEM,42,33.0,2.0
6v1xdL,1,MASC,31,8.0,68.0
ezRKm9,1,FEM,36,12.0,1.0
1Q35ej,1,MASC,69,0.0,0.0
EAN4J6,1,FEM,32,3.0,1.0


In [53]:
kaggle_postulantes['hay_datos_educacion'].value_counts()

1    59929
0     4717
Name: hay_datos_educacion, dtype: int64

In [54]:
kaggle_postulantes['sexo'].value_counts()

MASC          31286
FEM           31037
NO_DECLARA     2323
Name: sexo, dtype: int64

In [55]:
kaggle_postulantes['edad'].describe()

count    64646.000000
mean        35.462766
std         11.073239
min          0.000000
25%         30.000000
50%         35.000000
75%         42.000000
max         81.000000
Name: edad, dtype: float64

In [56]:
kaggle_postulantes['cantidad_postulaciones'].describe()

count    64646.000000
mean        20.797404
std         56.902917
min          0.000000
25%          0.000000
50%          5.000000
75%         19.000000
max       3520.000000
Name: cantidad_postulaciones, dtype: float64

In [57]:
kaggle_postulantes['cantidad_vistas'].describe()

count    64646.000000
mean        42.354670
std         89.342421
min          0.000000
25%          4.000000
50%         14.000000
75%         44.000000
max       4443.000000
Name: cantidad_vistas, dtype: float64

In [149]:
mascara = lambda clave, min_c, max_c: (postulantes[clave] > min_c) & (postulantes[clave] <= max_c)

postulantes_a = postulantes[mascara('cantidad_postulaciones', -1, 0)]
postulantes_b = postulantes[mascara('cantidad_postulaciones', 0, 5)]
postulantes_c = postulantes[mascara('cantidad_postulaciones', 5, 19)]
postulantes_d = postulantes[mascara('cantidad_postulaciones', 19, 3520)]

postulantes_for_sample = \
    pd.DataFrame(postulantes[mascara('cantidad_vistas', -1, 4)].sample(n=25000)).append(
        pd.DataFrame(postulantes[mascara('cantidad_vistas', 4, 14)].sample(n=25000)).append(
            pd.DataFrame(postulantes[mascara('cantidad_vistas', 14, 44)].sample(n=25000)).append(
                pd.DataFrame(postulantes[mascara('cantidad_vistas', 44, 4443)].sample(n=25000))
            )
        )
    )

postulantes_for_sample['cantidad_postulaciones'].describe()

count    100000.000000
mean         17.168700
std          44.472391
min           0.000000
25%           0.000000
50%           5.000000
75%          17.000000
max        3520.000000
Name: cantidad_postulaciones, dtype: float64

In [150]:
postulantes_for_sample['cantidad_vistas'].describe()

count    100000.000000
mean         37.809520
std          72.989258
min           0.000000
25%           4.750000
50%          14.500000
75%          44.250000
max        4443.000000
Name: cantidad_vistas, dtype: float64

In [151]:
postulantes_for_sample['edad'].describe()

count    100000.000000
mean         29.807300
std          10.928338
min           0.000000
25%          24.000000
50%          29.000000
75%          36.000000
max          78.000000
Name: edad, dtype: float64

In [155]:
len(postulantes_for_sample.reset_index().drop_duplicates())

100000

In [78]:
postulantes_for_sample.head()

,Doctorado,Master,Otro,Posgrado,Secundario,Terciario/Técnico,Universitario,sexo,edad,cantidad_vistas,cantidad_postulaciones
idpostulante,,,,,,,,,,,
YVK45Y,0,0,0,0,0,0,0,FEM,55,0.0,0.0
dYjVe2A,0,0,0,0,Graduado,0,0,MASC,22,0.0,0.0
rmdap1A,0,0,0,0,0,0,Abandonado,FEM,37,4.0,0.0
0zPbW8Y,0,0,0,0,Graduado,0,0,FEM,20,0.0,0.0
QNr9x2N,0,0,0,0,Graduado,0,0,MASC,24,1.0,0.0


In [79]:
postulantes_for_sample.to_csv('/home/luciano/orga-datos/nuevo_set/postulantes_for_sample.csv', index=False)